## This part of code is  for using different learning rates  with non-normalized data and also
## generates output value for test data 
## Contains mostly question 3

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [4]:
main_df_non_normalized = pd.read_csv('PA1_train.csv.xls')

In [20]:
df_non_normalized = main_df_non_normalized
df_non_normalized = df_non_normalized.drop(['id'],1)
df_non_normalized['date'] = pd.to_datetime(main_df_non_normalized['date'])
df_non_normalized['month'] = df_non_normalized['date'].apply(lambda x:x.month)
df_non_normalized['year'] =  df_non_normalized['date'].apply(lambda x:x.year)
df_non_normalized['day'] = df_non_normalized['date'].apply(lambda x:x.day)
y_train = df_non_normalized['price']
df_non_normalized = df_non_normalized.drop(['date','price'],1)

In [21]:
df_non_normalized.head()

,dummy,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month,year,day
0,1,4,2.50,2720,10006,2.0,0,0,3,9,...,1989,0,98074,47.6295,-122.042,2720,10759,7,2014,9
1,1,2,2.50,2600,5000,1.0,0,0,5,8,...,1926,0,98126,47.5806,-122.379,2260,5000,7,2014,18
2,1,5,1.75,1650,3000,1.5,0,0,3,8,...,1902,0,98144,47.5955,-122.306,1740,4000,7,2014,7
3,1,4,1.75,1720,9600,1.0,0,0,4,8,...,1969,0,98059,47.4764,-122.155,1660,10720,4,2015,28
4,1,6,2.25,3830,11180,1.0,0,2,5,9,...,1962,0,98008,47.5849,-122.113,2500,10400,9,2014,4


In [22]:
df_non_normalized.columns

Index(['dummy', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'month', 'year', 'day'],
      dtype='object')

In [23]:
df_non_normalized.shape

(10000, 22)

In [24]:
def check_explode(norm):
    does_explode = True
    for i in range(1,len(norm)-1):
        if norm[i]<norm[i-1]:
            does_explode = False
    return does_explode

In [29]:
all_info={}
import math
def calculate_weights_and_sse(learning_rate,check_iter_for_explode,max_iteration=10000,df=df_non_normalized):
    
    weight = np.array(np.random.randn(1,22)[0])
    
    number_of_iteration = 0
    sse_for_each_iter = [] 
    norm_for_this_lr = []
    while True:
        diff_in_y = df.dot(weight)-y_train
        
        
        gradient = (np.transpose(df)).dot(diff_in_y)
        
        weight = weight - learning_rate*gradient
        
        sse = (np.transpose(diff_in_y)).dot(diff_in_y)
        
        
        norm_for_gradient = math.sqrt(gradient.dot(gradient))
        
        
        #print('number of iteration',number_of_iteration,norm_for_gradient,'   sse =',sse)
        
        
        sse_for_each_iter.append(sse)
        
        norm_for_this_lr.append(norm_for_gradient)
        
        
        number_of_iteration = number_of_iteration + 1
        
        
        if(norm_for_gradient<=0.5):
            
            all_info[learning_rate] = {"whether converged":True,
                               "weights":weight,
                               "sse_for_each_iteration":sse_for_each_iter,
                               "number_of_iteration_used":number_of_iteration
                              }
            print("########################")
            print("number of itertion used for learning_rate =",learning_rate , " is ",number_of_iteration , 'and sse is ' ,sse)
            print("########################")
            break
            
            
        if(number_of_iteration == check_iter_for_explode and check_explode(norm_for_this_lr)):
            
            print('Gradient explodes for learning_rate =',learning_rate)
            all_info[learning_rate] = {"whether converged":False,
                               "weights":np.array(np.zeros(12)),
                               "sse_for_each_iteration":np.array([-1]*12),
                               "number_of_iteration_used":number_of_iteration
                              }
            break
            
        if(number_of_iteration==max_iteration):
    
            all_info[learning_rate] = {"whether converged":True,
                               "weights":weight,
                               "sse_for_each_iteration":sse_for_each_iter,
                               "number_of_iteration_used":number_of_iteration
                              }
            print("converges but takes too many iterations  for lambda =",learning_rate)
            break
            
            
        if (number_of_iteration%5000)==0:
            print('number of iteration',number_of_iteration,norm_for_gradient,sse)
    
    print('checked for learning rate = ',learning_rate," now returning")
    return
        
            


`learning rate zero implies that we randomly assume any weight for predicting the model as there willl be no contribution of gradient.
Weights will be constant in the minimization process `

In [30]:
learning_rate = [1,0.001,10**(-6),10**(-9),10**(-15)]
for l in learning_rate:
    calculate_weights_and_sse(l,20)

Gradient explodes for learning_rate = 1
checked for learning rate =  1  now returning
Gradient explodes for learning_rate = 0.001
checked for learning rate =  0.001  now returning
Gradient explodes for learning_rate = 1e-06
checked for learning rate =  1e-06  now returning
Gradient explodes for learning_rate = 1e-09
checked for learning rate =  1e-09  now returning
number of iteration 5000 33258366274.00269 73674995379.21494
converges but takes too many iterations  for lambda = 1e-15
checked for learning rate =  1e-15  now returning


In [31]:
all_info.keys()

dict_keys([1, 0.001, 1e-06, 1e-09, 1e-15])

In [33]:
all_info[(10**(-15))]['weights']

dummy           -2.899204
bedrooms        -1.109639
bathrooms        0.905684
sqft_living      1.698675
sqft_lot        -0.003205
floors          -0.717336
waterfront       0.614414
view            -0.552092
condition       -0.034274
grade           -1.502894
sqft_above       0.278532
sqft_basement    1.379978
yr_built        -0.435759
yr_renovated     1.264413
zipcode         -0.053202
lat             -0.075168
long             0.381110
sqft_living15    0.851915
sqft_lot15      -0.012508
month            0.675364
year             0.051845
day              1.081190
dtype: float64

In [35]:
all_info[(10**(-15))]['sse_for_each_iteration'][9999]

63481517003.65262

In [37]:
main_test_df = pd.read_csv('PA1_test.csv.xls')

In [39]:
test_df = main_test_df
test_df= test_df.drop(['id'],1)
test_df['date'] = pd.to_datetime(main_test_df['date'])
test_df['month'] = test_df['date'].apply(lambda x:x.month)
test_df['year'] =  test_df['date'].apply(lambda x:x.year)
test_df['day'] = test_df['date'].apply(lambda x:x.day)
test_df = test_df.drop(['date'],1)

In [41]:
test_df.head()

,dummy,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month,year,day
0,1,3,1.75,1800,8000,1.0,0,0,3,8,...,1972,0,98008,47.6357,-122.119,1950,8500,9,2014,11
1,1,2,1.50,1110,957,2.0,0,0,3,8,...,2005,0,98119,47.6319,-122.362,1680,1104,5,2014,8
2,1,4,2.50,1700,6675,2.0,0,0,3,7,...,1997,0,98038,47.3638,-122.053,1570,7540,4,2015,8
3,1,4,2.75,2760,7533,1.0,0,0,3,8,...,1997,0,98042,47.3710,-122.108,2340,7943,11,2014,10
4,1,5,1.75,2640,7722,1.5,0,0,4,7,...,1915,0,98117,47.6802,-122.361,1750,4000,3,2015,23


In [43]:
test_df.shape

(6000, 22)

In [44]:
test_df = test_df.drop(['dummy'],1)

In [45]:
test_df.shape

(6000, 21)

In [46]:
for feature in test_df.columns:
    mn = test_df[feature].min()
    rnge = test_df[feature].max() - test_df[feature].min()
    test_df[feature] = test_df[feature].apply(lambda x : (x-mn)/rnge)

In [47]:
test_df.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month,year,day
0,0.222222,0.166667,0.121680,0.006425,0.0,0.0,0.0,0.50,0.444444,0.173382,...,0.626087,0.0,0.035354,0.770624,0.332226,0.287702,0.018847,0.727273,0.0,0.333333
1,0.111111,0.133333,0.062554,0.000375,0.4,0.0,0.0,0.50,0.444444,0.067155,...,0.913043,0.0,0.595960,0.764478,0.130399,0.237618,0.000861,0.363636,0.0,0.233333
2,0.333333,0.266667,0.113111,0.005287,0.4,0.0,0.0,0.50,0.333333,0.161172,...,0.843478,0.0,0.186869,0.330799,0.387043,0.217214,0.016512,0.272727,1.0,0.233333
3,0.333333,0.300000,0.203942,0.006023,0.0,0.0,0.0,0.50,0.444444,0.124542,...,0.843478,0.0,0.207071,0.342446,0.341362,0.360045,0.017492,0.909091,0.0,0.300000
4,0.444444,0.166667,0.193659,0.006186,0.2,0.0,0.0,0.75,0.333333,0.155067,...,0.130435,0.0,0.585859,0.842608,0.131229,0.250603,0.007903,0.181818,1.0,0.733333


In [58]:
weight_vector = pd.read_csv('weight_regular_0.001.csv')

In [59]:
weight_vector

,weight,feature
0,0.333747,1
1,3.250678,2
2,4.820770,3
3,0.033134,4
4,1.108012,5
5,2.259873,6
6,3.413060,7
7,0.275100,8
8,7.240978,9
9,4.114920,10


In [60]:
weight_vector = np.array(weight_vector['weight'])

In [61]:

print(test_df.shape,weight_vector.shape)

(6000, 21) (21,)


In [62]:
y_test = test_df.dot(weight_vector)

In [63]:
y_test.head()

0    5.349723
1    3.927446
2    2.724960
3    4.270439
4    7.096441
dtype: float64

In [64]:
y_test.to_csv('output.csv',index=False)